In [79]:
import datetime as dt
import pandas as pd
pd.set_option('display.max_columns', None)

In [81]:
df = pd.read_excel("online_retail_II.xlsx", sheet_name="Year 2010-2011")


In [83]:
df.head()
# Number of NaN values per column:
print(df.isnull().sum())


Invoice             0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
Price               0
Customer ID    135080
Country             0
dtype: int64


In [85]:
# Convert Quantity to numeric, coercing errors to NaN
df['Quantity'] = pd.to_numeric(df['Quantity'], errors='coerce')

In [87]:
# Drop rows with NaN values in any column
df.dropna(inplace=True)

In [89]:
# Filter out rows with negative Quantity and invoices containing "C"
df = df[(df['Quantity'] > 0) & (~df['Invoice'].str.contains("C", na=False))]

In [91]:
# Describe with specified percentiles
description = df.describe([0.01, 0.25, 0.50, 0.75, 0.99]).T
print(description)

                count                           mean                  min  \
Quantity     397925.0                      13.021793                  1.0   
InvoiceDate    397925  2011-07-10 23:44:09.817126400  2010-12-01 08:26:00   
Price        397925.0                       3.116212                  0.0   
Customer ID  397925.0                   15294.308601              12346.0   

                              1%                  25%                  50%  \
Quantity                     1.0                  2.0                  6.0   
InvoiceDate  2010-12-03 10:26:00  2011-04-07 11:12:00  2011-07-31 14:39:00   
Price                       0.21                 1.25                 1.95   
Customer ID              12415.0              13969.0              15159.0   

                             75%                  99%                  max  \
Quantity                    12.0                120.0              80995.0   
InvoiceDate  2011-10-20 14:33:00  2011-12-07 14:01:00  2011-12-09 1

In [93]:
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit

In [95]:
# Define the function to replace outliers with thresholds
def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit

In [97]:
# Define your outlier_thresholds function (assuming it's already defined elsewhere in your script)
def outlier_thresholds(dataframe, variable):
    # Implement your logic to calculate low_limit and up_limit for outlier thresholds
    # Example:
    low_limit = dataframe[variable].quantile(0.01)
    up_limit = dataframe[variable].quantile(0.99)
    return low_limit, up_limit

In [99]:
# Now, let's calculate RFM metrics and segmentation

# Calculate TotalPrice
df["TotalPrice"] = df["Quantity"] * df["Price"]

# Set today's date
today_date = dt.datetime(2011, 12, 11)

# Calculate Recency, Frequency, and Monetary metrics for RFM analysis
rfm = df.groupby("Customer ID").agg({
    "InvoiceDate": lambda date: (today_date - date.max()).days,
    "Invoice": lambda num: num.nunique(),
    "TotalPrice": lambda price: price.sum()
})

In [101]:
# Rename columns for clarity
rfm.columns = ['Recency', 'Frequency', "Monetary"]

# Reset index to bring 'Customer ID' back as a column
rfm.reset_index(inplace=True)


In [103]:
# Calculate RFM scores
rfm["RecencyScore"] = pd.qcut(rfm["Recency"], 5, labels=[5, 4, 3, 2, 1])
rfm["FrequencyScore"] = pd.qcut(rfm["Frequency"].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])
rfm["MonetaryScore"] = pd.qcut(rfm["Monetary"], 5, labels=[1, 2, 3, 4, 5])

In [105]:
# Combine RFM scores into RFM_SCORE as a concatenated string
rfm["RFM_SCORE"] = (rfm['RecencyScore'].astype(str) +
                    rfm['FrequencyScore'].astype(str) +
                    rfm['MonetaryScore'].astype(str))

In [107]:
# Define segment mapping based on RFM_SCORE ranges
seg_map = {
    r'[1-2][1-2]': 'Hibernating',
    r'[1-2][3-4]': 'At_Risk',
    r'[1-2]5': 'Cant_Loose',
    r'3[1-2]': 'About_to_Sleep',
    r'33': 'Need_Attention',
    r'[3-4][4-5]': 'Loyal_Customers',
    r'41': 'Promising',
    r'51': 'New_Customers',
    r'[4-5][2-3]': 'Potential_Loyalists',
    r'5[4-5]': 'Champions'
}

In [109]:
# Create 'Segment' based on RecencyScore and FrequencyScore using seg_map
rfm['Segment'] = rfm['RecencyScore'].astype(str) + rfm['FrequencyScore'].astype(str)
rfm['Segment'] = rfm['Segment'].replace(seg_map, regex=True)

In [143]:

# Function to find segment for a specific Customer ID
def find_customer_segment(customer_id):
    try:
        segment = rfm.loc[rfm["Customer ID"] == customer_id, "Segment"].values[0]
        return segment
    except IndexError:
        return "Customer ID not found in the dataset"


if __name__ == "__main__":
    try:
        customer_id = float(input("Enter Customer ID: "))
        segment = find_customer_segment(customer_id)
        print(f"Segment for Customer ID {customer_id}: {segment}")
    except ValueError:
        print("Please enter a valid Customer ID")

Enter Customer ID:  12345


Segment for Customer ID 12345.0: Customer ID not found in the dataset


In [145]:
# Now, find the segment for a specific Customer ID
Customer_ID = 12347.0
customer_segment = rfm[rfm["Customer ID"] == Customer_ID]["Segment"].values[0]

# Print the segment for the specified Customer ID
print(f"Segment for Customer ID {Customer_ID}: {customer_segment}") 

Segment for Customer ID 12347.0: Champions
